# Change in number of language speakers in NYC between 2005 and 2019
Using data from the American Community Survey pulled from the API in the languages-2005 and -2019 spreadsheets

In [1]:
import pandas as pd
import numpy as np

/Users/ilenapeng/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Load data

In [2]:
# https://api.census.gov/data/2005/acs/acs1/groups/B16001.html
df_05 = pd.read_csv('data/languages-2005.csv')
df_05['year'] = '2005'
df_05.head(2)

,county,borough,language,speaker_num,lang_family,year
0,5,Bronx,Spanish,556951.0,Indo-European,2005
1,47,Brooklyn,Spanish,398837.0,Indo-European,2005


In [3]:
# https://api.census.gov/data/2010/acs/acs1/groups/B16001.html
df_10 = pd.read_csv('data/languages-2010.csv')
df_10['year'] = '2010'
df_10.head(2)

,county,borough,language,speaker_num,lang_family,year
0,5,Bronx,Spanish,603543.0,Indo-European,2010
1,47,Brooklyn,Spanish,401318.0,Indo-European,2010


In [4]:
# https://api.census.gov/data/2015/acs/acs1/groups/B16001.html
df_15 = pd.read_csv('data/languages-2015.csv')
df_15['year'] = '2015'
df_15.head(2)

,county,borough,language,speaker_num,lang_family,year
0,81,Queens,Spanish,524568,Indo-European,2015
1,61,Manhattan,Spanish,340566,Indo-European,2015


In [5]:
# https://api.census.gov/data/2019/acs/acs1/groups/B16001.html
df_19 = pd.read_csv('data/languages-2019.csv')
df_19['year'] = '2019'
df_19.head(2)

,county,borough,language,speaker_num,lang_family,year
0,5,Bronx,Spanish,631573,Indo-European,2019
1,47,Brooklyn,Spanish,363599,Indo-European,2019


## Combine data and address inconsistencies across datasets

In [6]:
# stack dataframes
df_stack = pd.concat([df_05, df_10, df_15, df_19])
df_stack.year.unique()

array(['2005', '2010', '2015', '2019'], dtype=object)

In [7]:
# Doing some grouping to lend some consistency across datasets
# Replace Yiddish with Other Germanic
df_stack['language'] = df_stack.language.str.replace('Yiddish', 'Other Germanic', regex=False)

# Replace Cambodian with Mon-Khmer
df_stack['language'] = df_stack.language.str.replace('Cambodian', 'Mon-Khmer', regex=False)

# Replace Haitian with Haitian/French Creole
df_stack['language'] = df_stack.language.str.replace('Haitian', 'Haitian and French Creole', regex=False)

# Replace Other West African with Other African
df_stack['language'] = df_stack.language.str.replace('Other West African', 'Other African', regex=False)

# Replace Other Native North American with Other Native American
df_stack['language'] = df_stack.language.str.replace('ther Native North American', 'ther Native American', regex=False)

In [8]:
# Do a groupby so that West African and African get added together
df_stack = df_stack.groupby(['language', 'year', 'borough', 'county', 'lang_family']).speaker_num.sum().reset_index()
df_stack.head()

,language,year,borough,county,lang_family,speaker_num
0,Arabic,2005,Bronx,5,Other,4398.0
1,Arabic,2005,Brooklyn,47,Other,27351.0
2,Arabic,2005,Manhattan,61,Other,6327.0
3,Arabic,2005,Queens,81,Other,12865.0
4,Arabic,2005,Staten Island,85,Other,0.0


In [9]:
# replace 0s with NaNs
df_stack['speaker_num'] = df_stack.speaker_num.replace(0, np.nan)
df_stack.head()

,language,year,borough,county,lang_family,speaker_num
0,Arabic,2005,Bronx,5,Other,4398.0
1,Arabic,2005,Brooklyn,47,Other,27351.0
2,Arabic,2005,Manhattan,61,Other,6327.0
3,Arabic,2005,Queens,81,Other,12865.0
4,Arabic,2005,Staten Island,85,Other,NaN


In [10]:
df_stack.to_csv('data/languages-over-time-long.csv', index=False)

In [11]:
# reshape
df = df_stack.pivot(index=['county', 'borough', 'lang_family', 'language'], columns='year', values='speaker_num').reset_index()
df.head()

year,county,borough,lang_family,language,2005,2010,2015,2019
0,5,Bronx,Asian and Pacific Island,Chinese,7902.0,7359.0,7550.0,5420.0
1,5,Bronx,Asian and Pacific Island,Hmong,NaN,NaN,NaN,NaN
2,5,Bronx,Asian and Pacific Island,Japanese,264.0,390.0,500.0,358.0
3,5,Bronx,Asian and Pacific Island,Khmer,NaN,NaN,NaN,1292.0
4,5,Bronx,Asian and Pacific Island,Korean,4308.0,2669.0,2279.0,1455.0


## Calculate change between 2005 and 2019

In [12]:
# calculate difference between two years
df['difference'] = df['2019'] - df['2005']
df.head(5)

year,county,borough,lang_family,language,2005,2010,2015,2019,difference
0,5,Bronx,Asian and Pacific Island,Chinese,7902.0,7359.0,7550.0,5420.0,-2482.0
1,5,Bronx,Asian and Pacific Island,Hmong,NaN,NaN,NaN,NaN,NaN
2,5,Bronx,Asian and Pacific Island,Japanese,264.0,390.0,500.0,358.0,94.0
3,5,Bronx,Asian and Pacific Island,Khmer,NaN,NaN,NaN,1292.0,NaN
4,5,Bronx,Asian and Pacific Island,Korean,4308.0,2669.0,2279.0,1455.0,-2853.0


In [13]:
# create column with percent change
df['percent_change'] = ((df['2019']-df['2005'])/df['2005'])*100
df.head(5)

year,county,borough,lang_family,language,2005,2010,2015,2019,difference,percent_change
0,5,Bronx,Asian and Pacific Island,Chinese,7902.0,7359.0,7550.0,5420.0,-2482.0,-31.409770
1,5,Bronx,Asian and Pacific Island,Hmong,NaN,NaN,NaN,NaN,NaN,NaN
2,5,Bronx,Asian and Pacific Island,Japanese,264.0,390.0,500.0,358.0,94.0,35.606061
3,5,Bronx,Asian and Pacific Island,Khmer,NaN,NaN,NaN,1292.0,NaN,NaN
4,5,Bronx,Asian and Pacific Island,Korean,4308.0,2669.0,2279.0,1455.0,-2853.0,-66.225627


In [14]:
# create column with positive or negative change
# df.loc[df.difference > 0, 'change']='Positive'
# df.loc[df.difference < 0, 'change']='Negative'
# df.head()

In [15]:
df.to_csv('data/languages-over-time-wide.csv', index=False)

## Look at languages in Brooklyn

In [20]:
df_bklyn = df[df.borough=='Brooklyn']
df_bklyn.borough.unique()

array(['Brooklyn'], dtype=object)

In [22]:
df_bklyn.head()

year,county,borough,lang_family,language,2005,2010,2015,2019,difference,percent_change
47,47,Brooklyn,Asian and Pacific Island,Chinese,125123.0,163961.0,181712.0,173022.0,47899.0,38.281531
48,47,Brooklyn,Asian and Pacific Island,Hmong,NaN,NaN,NaN,62.0,NaN,NaN
49,47,Brooklyn,Asian and Pacific Island,Japanese,2581.0,3549.0,4918.0,3322.0,741.0,28.709802
50,47,Brooklyn,Asian and Pacific Island,Khmer,NaN,NaN,NaN,265.0,NaN,NaN
51,47,Brooklyn,Asian and Pacific Island,Korean,3604.0,4744.0,4627.0,5921.0,2317.0,64.289678


In [33]:
# Languages with highest drop in raw numbers
num_dec_count = df_bklyn.sort_values('difference', ascending=True).head(5)
num_dec_count

year,county,borough,lang_family,language,2005,2010,2015,2019,difference,percent_change
83,47,Brooklyn,Indo-European,Spanish,398837.0,401318.0,399775.0,363599.0,-35238.0,-8.835188
70,47,Brooklyn,Indo-European,Italian,31917.0,26724.0,20010.0,16555.0,-15362.0,-48.131090
81,47,Brooklyn,Indo-European,Russian,137872.0,128288.0,128359.0,123340.0,-14532.0,-10.540211
78,47,Brooklyn,Indo-European,Polish,26911.0,25269.0,19956.0,13089.0,-13822.0,-51.361897
73,47,Brooklyn,Indo-European,Other Indic,14490.0,19642.0,24167.0,2610.0,-11880.0,-81.987578


In [34]:
# Languages with greatest increase in raw numbers
num_inc_count = df_bklyn.sort_values('difference', ascending=False).head(5)
num_inc_count

year,county,borough,lang_family,language,2005,2010,2015,2019,difference,percent_change
47,47,Brooklyn,Asian and Pacific Island,Chinese,125123.0,163961.0,181712.0,173022.0,47899.0,38.281531
86,47,Brooklyn,Indo-European,Urdu,9481.0,13557.0,21199.0,19384.0,9903.0,104.451007
54,47,Brooklyn,Asian and Pacific Island,Other Asian,7217.0,9929.0,14820.0,13704.0,6487.0,89.884994
91,47,Brooklyn,Other,Other African,9193.0,14538.0,14812.0,12570.0,3377.0,36.734472
87,47,Brooklyn,Other,Arabic,27351.0,22154.0,29893.0,30680.0,3329.0,12.171401


In [40]:
# Combine dataframes and export for diverging bar chart
bklyn_inc_dec = pd.concat([num_dec_count, num_inc_count])
bklyn_inc_dec.to_csv('data/bklyn-top-lang-differences.csv', index=False)

In [35]:
# Languages with highest percent change drop
pct_dec_count = df_bklyn.sort_values('percent_change', ascending=True).head(5)
pct_dec_count

year,county,borough,lang_family,language,2005,2010,2015,2019,difference,percent_change
73,47,Brooklyn,Indo-European,Other Indic,14490.0,19642.0,24167.0,2610.0,-11880.0,-81.987578
82,47,Brooklyn,Indo-European,Serbo-Croatian,3315.0,2276.0,3296.0,1467.0,-1848.0,-55.746606
67,47,Brooklyn,Indo-European,Gujarati,2821.0,1057.0,1132.0,1319.0,-1502.0,-53.243531
78,47,Brooklyn,Indo-European,Polish,26911.0,25269.0,19956.0,13089.0,-13822.0,-51.361897
70,47,Brooklyn,Indo-European,Italian,31917.0,26724.0,20010.0,16555.0,-15362.0,-48.131090


In [36]:
# Languages with greatest percent change drop
pct_inc_count = df_bklyn.sort_values('difference', ascending=False).head(5)
pct_inc_count

year,county,borough,lang_family,language,2005,2010,2015,2019,difference,percent_change
47,47,Brooklyn,Asian and Pacific Island,Chinese,125123.0,163961.0,181712.0,173022.0,47899.0,38.281531
86,47,Brooklyn,Indo-European,Urdu,9481.0,13557.0,21199.0,19384.0,9903.0,104.451007
54,47,Brooklyn,Asian and Pacific Island,Other Asian,7217.0,9929.0,14820.0,13704.0,6487.0,89.884994
91,47,Brooklyn,Other,Other African,9193.0,14538.0,14812.0,12570.0,3377.0,36.734472
87,47,Brooklyn,Other,Arabic,27351.0,22154.0,29893.0,30680.0,3329.0,12.171401


## Calculate total count of all language speakers in each borough

In [16]:
df_borough_sum = df_stack.groupby(['year', 'borough', 'county']).speaker_num.sum().reset_index()
# sort by borough, not by year
df_borough_sum = df_borough_sum.sort_values('borough')
df_borough_sum.head()

,year,borough,county,speaker_num
0,2005,Bronx,5,679603.0
5,2010,Bronx,5,734618.0
10,2015,Bronx,5,808881.0
15,2019,Bronx,5,777475.0
1,2005,Brooklyn,47,1038733.0


In [17]:
# replace 0s with NaNs
df_borough_sum['speaker_num'] = df_borough_sum.speaker_num.replace(0, np.nan)
df_stack.head()

,language,year,borough,county,lang_family,speaker_num
0,Arabic,2005,Bronx,5,Other,4398.0
1,Arabic,2005,Brooklyn,47,Other,27351.0
2,Arabic,2005,Manhattan,61,Other,6327.0
3,Arabic,2005,Queens,81,Other,12865.0
4,Arabic,2005,Staten Island,85,Other,NaN


In [18]:
# reshape wide to long
df_borough_sum = df_borough_sum.pivot(index=['county', 'borough'], columns='year', values='speaker_num').reset_index()

# make difference variable
df_borough_sum['difference'] = df_borough_sum['2019'] - df_borough_sum['2005']

# make percent change variable
df_borough_sum['percent_change'] = ((df_borough_sum['2019']-df_borough_sum['2005'])/df_borough_sum['2005'])*100

df_borough_sum.head()

year,county,borough,2005,2010,2015,2019,difference,percent_change
0,5,Bronx,679603.0,734618.0,808881.0,777475.0,97872.0,14.401349
1,47,Brooklyn,1038733.0,1076912.0,1134560.0,960155.0,-78578.0,-7.564793
2,61,Manhattan,544990.0,608868.0,621654.0,582109.0,37119.0,6.810951
3,81,Queens,1098848.0,1204425.0,1239897.0,1174485.0,75637.0,6.883300
4,85,Staten Island,NaN,NaN,136924.0,151764.0,NaN,NaN


In [19]:
df_borough_sum.to_csv('data/language-borough-totals-over-time.csv', index=False)